# See Inverse dynamics regression

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
#%config Completer.use_jedi = False

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import (plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions)
from vct.bokeh_plotting import create_tab
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify
plt.style.use('bmh')
plt.rcParams["figure.figsize"] = (15,10)
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
import statsmodels.api as sm

In [ ]:
#%reload_kedro
ship='wPCC'
ship_data = catalog.load(f"{ship}.ship_data")

models_ID_hull = catalog.load(f"{ship}.models_ID_hull")
models_ID_hull_rudder = catalog.load(f"{ship}.models_ID_hull_rudder")
models_VCT_MDL_optimize = catalog.load(f"{ship}.models_VCT_MDL_optimize")
models_VCT_MDL = catalog.load(f"{ship}.models_VCT_MDL")


df_VCT = catalog.load(f"{ship}.df_VCT")
resistance_MDL = catalog.load(f"{ship}.resistance_MDL")
#tests_ek_smooth = catalog.load(f"{ship}.tests_ek_smooth")
tests_ek = catalog.load(f"{ship}.tests_ek")

mask = ((df_VCT['Y_R_port'] > 0) & (df_VCT['test type'] == 'self propulsion'))
df_VCT = df_VCT.loc[~mask].copy()  ## remove duplicate self propulsion

time_series_meta_data = catalog.load(f"{ship}.time_series_meta_data")
#tests_ek_smooth_joined = catalog.load(f"{ship}.tests_ek_smooth_joined")
tests_ek_joined = catalog.load(f"{ship}.tests_ek_joined")

df_VCT = catalog.load(f"{ship}.df_VCT")
mask = ((df_VCT['Y_R_port'] > 0) & (df_VCT['test type'] == 'self propulsion'))
df_VCT = df_VCT.loc[~mask].copy()  ## remove duplicate self propulsion


In [ ]:
fig,ax=plt.subplots()

mask = tests_ek_joined['id'].isin([
    #22774,  # Circle
    #22611,
    #22635,
    #22639,
    22769,
    22771,
])

tests_ek_smooth_joined = tests_ek_joined.loc[~mask].copy()

for id, group in tests_ek_smooth_joined.groupby(by='id'):
    group[['x0','y0']].rolling(window=100).mean().plot(x='y0',y='x0', label=id, ax=ax)
ax.axis('equal')

In [ ]:
model_ID_hull = models_ID_hull['semiempirical_covered']()

In [ ]:
model_ID_hull_rudder = models_ID_hull_rudder['Abkowitz']()

In [ ]:
model_VCT_MDL_optimize = models_VCT_MDL_optimize['semiempirical_covered']()

In [ ]:
model_VCT_MDL = models_VCT_MDL['semiempirical_covered']()

In [ ]:
#id = 22773
#id = 22772
id = 22770
#id = 22764
#id = 22774
#id = 22616  # Spiral


data_MDL = tests_ek[f'{id}']()
data_MDL['V'] = data_MDL['U'] = np.sqrt(data_MDL['u']**2 + data_MDL['v']**2)
data_MDL['beta'] = -np.arctan2(data_MDL['v'],data_MDL['u'])
data_MDL['rev'] = data_MDL[['Prop/PS/Rpm','Prop/SB/Rpm']].mean(axis=1)
data_MDL['twa']=0
data_MDL['tws']=0
data_MDL['theta']=0
data_MDL['q']=0
data_MDL['phi'] = data_MDL['roll']
data_MDL['p'] = 0
data_MDL['q1d'] = 0
data_MDL['thrust_port'] = data_MDL['Prop/PS/Thrust']
data_MDL['thrust_stbd'] = data_MDL['Prop/SB/Thrust']
data_MDL['beta_deg'] =  np.rad2deg(data_MDL['beta'])

In [ ]:
styles = {
    "Experiment": {
        "style": "-",
        "color": "black",
        "zorder": -10,
        "lw": 1.0,
        "label": "Experiment",
    },
    "VCT": {
        "style": "-",
        "color": "green",
        "zorder": -10,
        "lw": 1.0,
        "label": "VCT",
    },
    "VCT optimized": {
        "style": "-",
        "color": "red",
        "zorder": -10,
        "lw": 1.0,
        "label": "VCT optimized",
    },
    "ID hull": {
        "style": "-",
        "color": "blue",
        "zorder": -10,
        "lw": 1.0,
        "label": "ID hull",
    },
    "ID hull rudder": {
        "style": "-",
        "color": "magenta",
        "zorder": -10,
        "lw": 1.0,
        "label": "ID hull rudder",
    },
}


models = {
    'VCT' : model_VCT_MDL,
    'VCT optimized' : model_VCT_MDL_optimize,
    'ID hull' : model_ID_hull,
    'ID hull rudder' : model_ID_hull_rudder,
}

In [ ]:
plot_compare_model_forces(model=models, data=data_MDL, styles=styles, keys=['Y_D','Y_H','Y_R'])

In [ ]:
plot_compare_model_forces(model=models, data=data_MDL, styles=styles, keys=['N_D','N_H','N_R'])

In [ ]:
plot_compare_model_forces(model=models, data=data_MDL, styles=styles, keys=['X_D','X_H','X_R'])

In [ ]:
dataframes = {
    'Experiment':data_MDL,
}
for name, model in models.items():
    dataframes[name] = model.simulate(data_MDL)

In [ ]:
ax = track_plots(dataframes=dataframes, lpp=ship_data['L'], beam=ship_data['B'], styles=dict(styles), N=2);
plot(dataframes=dataframes, keys=['u','thrust','r','beta','delta','psi','N_R'], styles=dict(styles));

In [ ]:
#def plot_parameter_contributions(model, ax=None, prefix='N', unit='moment'):
#
#    if ax is None:
#        fig,ax=plt.subplots()
#    
#    hull = model.subsystems['hull']
#    df_parameters_contributions = hull.calculate_parameter_contributions(eq=hull.equations[f'{prefix}_H'], data=data_MDL, unit=unit)
#    
#    required = [f'{prefix}vv',f'{prefix}vvv',f'{prefix}rr',f'{prefix}rrr']
#    for item in required:
#        if not item in df_parameters_contributions:
#            df_parameters_contributions[item]=0
#    
#    df_parameters_contributions_clean = df_parameters_contributions.copy()
#    df_parameters_contributions_clean[f'{prefix}v+{prefix}vv+{prefix}vvv'] = df_parameters_contributions_clean[f'{prefix}v'] +  df_parameters_contributions_clean[f'{prefix}vv'] +  df_parameters_contributions_clean[f'{prefix}vvv']
#    df_parameters_contributions_clean[f'{prefix}r+{prefix}rr+{prefix}rrr'] = df_parameters_contributions_clean[f'{prefix}r'] +  df_parameters_contributions_clean[f'{prefix}rr'] +  df_parameters_contributions_clean[f'{prefix}rrr']
#    df_parameters_contributions_clean[f'{prefix}vvr+{prefix}vrr'] = df_parameters_contributions_clean[f'{prefix}vvr'] +  df_parameters_contributions_clean[f'{prefix}vrr']
#    df_parameters_contributions_clean.drop(columns=[f'{prefix}v',f'{prefix}vv',f'{prefix}r',f'{prefix}rr',f'{prefix}vvr',f'{prefix}vrr',f'{prefix}vvv',f'{prefix}rrr',f'{prefix}0'],inplace=True)
#    return df_parameters_contributions_clean.plot(ax=ax)

In [ ]:
fig,axes=plt.subplots(nrows=len(models))

for ax, (name, model) in zip(axes,models.items()):
                      
    ax.set_title(name)
    plot_parameter_contributions(model=model, data=data_MDL, ax=ax, prefix='N', unit='moment')

ylims = []
for ax in axes:
    ylims.append(ax.get_ylim())
    
ylims = [np.min(ylims, axis=0)[0],np.max(ylims, axis=0)[1]]
for ax in axes:
    ax.set_ylim(ylims) 

In [ ]:
fig,axes=plt.subplots(nrows=len(models))

for ax, (name, model) in zip(axes,models.items()):
                      
    ax.set_title(name)
    plot_parameter_contributions(model=model, data=data_MDL, ax=ax, prefix='Y', unit='force')

ylims = []
for ax in axes:
    ylims.append(ax.get_ylim())
    
ylims = [np.min(ylims, axis=0)[0],np.max(ylims, axis=0)[1]]
for ax in axes:
    ax.set_ylim(ylims) 

In [ ]:
_ = []
for name, model in models.items():
    _.append(pd.Series({key:value for key,value in model.parameters.items() if ((key[0]=='N') & ~('dot' in key) & ~('delta' in key))},name=name))
    
df_parameters_N = pd.concat(_,axis=1).transpose()

In [ ]:
df_parameters_N.transpose().plot.bar()

In [ ]:
prime_system_ship = PrimeSystem(L=ship_data['L']*ship_data['scale_factor'], rho=df_VCT.iloc[0]['rho'])
keys = ['x0',
        'y0',
        'psi',
        'u',
        'v',
        'r',
        'beta',
        'V',
        'U',
        'delta',
        'thrust',
        'thrust_port',
        'thrust_stbd',
        'twa',
        'tws',
        'rev',
        'X_D',
        'Y_D',
        'N_D',
        'X_H',
        'Y_H',
        'N_H',
        'X_R',
        'Y_R',
        'N_R',
        'X_R_port',
        'Y_R_port',
        'N_R_port',
        'X_R_stbd',
        'Y_R_stbd',
        'N_R_stbd',
        'fx',
        'fy',
        'mz',
        'test type',
        'model_name',
       ]



df_VCT_prime = prime_system_ship.prime(df_VCT[keys], U=df_VCT['U'])
df_VCT_scaled = model.prime_system.unprime(df_VCT_prime, U=df_VCT['U']/np.sqrt(ship_data['scale_factor']))

In [ ]:
_ = []
for name, model in models.items():
    df_VCT_predict = predict(model, data=df_VCT_scaled)
    df_VCT_predict['model_name'] = name
    _.append(df_VCT_predict)

df_result = pd.concat([df_VCT_scaled] + _, axis=0)

In [ ]:
df_result['model_name'].unique()

In [ ]:
create_tab(df_result, ys=['X_D','Y_D','N_D','X_R','Y_R','N_R'], plot_width=1200)

In [ ]:
model_ID_hull_rudder.show_subsystems()

In [ ]:
model_ID_hull.show_subsystems()